# Agentic AI with Docling as MCP tool

### Overview

In this lab, we will introduce **Docling MCP**, a service that provides tools for document conversion, processing, and generation. It also has extensions for leveraging popular agentic frameworks.

It uses the Docling library to convert PDF documents into structured formats and provides a caching mechanism to improve performance. The service exposes functionality through a set of tools that can be called by client applications.

---


### Technologies We'll Use

Building on our previous labs, we will leverage:

1. **[Docling](https://docling-project.github.io/docling/):** An open-source toolkit used to parse and convert documents.
2. **[MCP](https://modelcontextprotocol.io)**: The model context protocol for creating a tool.
3. **[Llama Stack](https://llama-stack.readthedocs.io/)**: Backend for building generative AI applications exposing standard APIs.
4. **[OpenAI Agents SDK](https://openai.github.io/openai-agents-python/)**: A lightweight framework for building agentic AI apps.

---

#### Notebook dependencies

Create a virtual environment to run this notebook, for instance, with [uv](https://docs.astral.sh/uv/), and install the necessary dependencies:

In [1]:
!uv pip install llama-stack-client openai-agents

Using Python 3.12.9 environment at: /Users/dol/projects/tx25/techxchange2025-lab3640-docling/.venv
Audited 2 packages in 14ms


In [2]:
import uuid
import logging

from rich.console import Console
from rich.markdown import Markdown

from agents import Agent, ModelSettings, Runner, SQLiteSession, set_trace_processors, set_tracing_disabled, ItemHelpers, FileSearchTool
from agents.mcp import MCPServerStreamableHttp
from agents.models.openai_provider import OpenAIProvider
from agents.run import RunConfig
from agents.items import ResponseFunctionToolCall, ResponseFileSearchToolCall
from agents.tracing.processors import BatchTraceProcessor, ConsoleSpanExporter
from openai import AsyncOpenAI
from llama_stack_client import LlamaStackClient

console = Console(width=100, soft_wrap=True)

In [3]:
LLS_URL = "http://localhost:8321"
BASE_URL = f"{LLS_URL}/v1/openai/v1"
API_KEY = "none"
MODEL_ID = "vllm/gpt-oss-120b"
# MODEL_ID = "vllm/llama-3-3-70b"
# MODEL_ID = "vllm/Llama-4-Maverick"

In [4]:
# Llama Stack Client
lls_client = LlamaStackClient(base_url=LLS_URL)

# Model client
client = AsyncOpenAI(base_url=BASE_URL, api_key=API_KEY)

# Configure the OpenAI provider that uses our AsyncOpenAI client for Llama Stack
provider = OpenAIProvider(openai_client=client)

# Tell OpenAI to dump traces to the console
set_tracing_disabled(True)
set_trace_processors([BatchTraceProcessor(exporter=ConsoleSpanExporter())])

# Setup quite logging
logging.getLogger("openai.agents").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)


In [5]:
instructions="""You are an assistant that uses external tools.  
Follow these rules strictly:

1. At each reasoning step, you may call **at most one tool**.  
2. If you need to use multiple tools, do it sequentially:  
   - Call exactly one tool.  
   - Wait for its result.  
   - Then think again and, if needed, call another tool.  
3. Never call two or more tools in the same response.  
4. If no tool is needed, just provide your reasoning or final answer.  
5. Treat a missing tool result as blocking — do not continue reasoning until you receive the output.

If you ever attempt to call more than one tool at once, your answer will be rejected.  
Always restrict yourself to **zero or one tool call per response**.

When the tools return a base64 image, simply create some markdown wrapper which allows to display it and take special care in not hallucinating any byte.
"""

### Agent definition

In the following block we define the Document Agent with:

1. Connection to the Docling MCP server
2. Selection of the tools to use in Docling
3. Possibly, use other tools, e.g. the built-in File Search

The `run_agent()` method will be called in the use cases below with different user prompts and tools.

In [6]:
async def run_agent(queries=[], extra_tools=[]):
    async with MCPServerStreamableHttp(
        name="Docling MCP",
        params={
            "url": "http://localhost:8000/mcp",
            "timeout": 60.0,
        },
        client_session_timeout_seconds=60,
    ) as server:
        agent = Agent(
            name="Document Agent",
            model=MODEL_ID,
            instructions=instructions,
            model_settings=ModelSettings(
                temperature=0, top_p=0.9,
                parallel_tool_calls=False,
                tool_choice="required",
            ),
            mcp_servers=[server],
            tools=extra_tools,
        )
        session = SQLiteSession(str(uuid.uuid4()))
        print(f"Created session_id={session.session_id} for Agent({agent.name})")

        user_queries = [instructions, *queries]

        for prompt in user_queries:
            console.print(f"[cyan]👤 User> {prompt}[/cyan]")
            # Launch the agent runner
            result = Runner.run_streamed(
                agent,
                prompt,
                session=session,
                run_config=RunConfig(model_provider=provider),
            )

            # Print the events as they appear from the agent stream
            async for event in result.stream_events():
                # We'll ignore the raw responses event deltas
                if event.type == "raw_response_event":
                    continue
                # When the agent updates, print that
                elif event.type == "agent_updated_stream_event":
                    console.print(f"Agent updated: {event.new_agent.name}")
                    continue
                # When items are generated, print them
                elif event.type == "run_item_stream_event":
                    if event.item.type == "tool_call_item":
                        raw_item = event.item.raw_item
                        if isinstance(raw_item, ResponseFunctionToolCall):
                            console.print(f"[yellow]-- Tool was called: {raw_item.name}({raw_item.arguments.strip()})[/yellow]")
                    elif event.item.type == "tool_call_output_item":
                        console.print(f"[yellow]-- Tool output: {event.item.output}[/yellow]")
                    elif event.item.type == "message_output_item":
                        md = Markdown(ItemHelpers.text_message_output(event.item))
                        console.print("[green]🤖 Assistant>[/green]")
                        console.print(md)
                    else:
                        print(f"other event: {event.item.type}")
                        pass  # Ignore other event types


---

## 1. Interact with documents

In this part of the lab, you will use the agent you just built to work with existing documents. The agent leverages both the reasoning capabilities of the LLM and the Docling MCP tools to interpret, process, and retrieve document content.

### Document conversion and exports

The Docling MCP tool allows to convert documents and return their content to the agent model for reasoning about it.
One of the classic use cases is document summarization.

In [7]:
await run_agent(
    queries=[
        "Convert the document on https://arxiv.org/pdf/2408.09869 and give me a summary of the document from its markdown content.",
    ],
    extra_tools=[],
)


INFO:mcp.client.streamable_http:Received session ID: 768960fd755c421480d49ebe53533144
INFO:mcp.client.streamable_http:Negotiated protocol version: 2025-06-18


Created session_id=5a0a9325-65fd-4459-b3c9-31cd9e0127c9 for Agent(Document Agent)


👤 User> You are an assistant that uses external tools.  
Follow these rules strictly:

1. At each reasoning step, you may call **at most one tool**.  
2. If you need to use multiple tools, do it sequentially:  
   - Call exactly one tool.  
   - Wait for its result.  
   - Then think again and, if needed, call another tool.  
3. Never call two or more tools in the same response.  
4. If no tool is needed, just provide your reasoning or final answer.  
5. Treat a missing tool result as blocking — do not continue reasoning until you receive the output.

If you ever attempt to call more than one tool at once, your answer will be rejected.  
Always restrict yourself to **zero or one tool call per response**.

When the tools return a base64 image, simply create some markdown wrapper which allows to display it and take special care in not hallucinating any byte.

Agent updated: Document Agent

🤖 Assistant>

...

-- Tool output: {"type":"text","text":"{\n  \"from_cache\": true,\n  \"document_key\": \"a66148ddcb5e4522d218e89ea3276db2\"\n}","annotations":null,"meta":null}

-- Tool output: {"type":"text","text":"{\n  \"document_key\": \"a66148ddcb5e4522d218e89ea3276db2\",\n  \"markdown\": \"<!-- image -->\\n\\n## Docling Technical Report\\n\\n## Version 1.0\\n\\nChristoph Auer Maksym Lysak Ahmed Nassar Michele Dolfi Nikolaos Livathinos Panos Vagenas Cesar Berrospi Ramis Matteo Omenetti Fabian Lindlbauer Kasper Dinkla Lokesh Mishra Yusik Kim Shubham Gupta Rafael Teixeira de Lima Valery Weber Lucas Morin Ingmar Meijer Viktor Kuropiatnyk Peter W. J. Staar\\n\\nAI4K Group, IBM Research R¨ uschlikon, Switzerland\\n\\n## Abstract\\n\\nThis technical report introduces Docling , an easy to use, self-contained, MITlicensed open-source package for PDF document conversion. It is powered by state-of-the-art specialized AI models for layout analysis (DocLayNet) and table structure recognition (TableFormer), and runs efficiently on commodity hardware in a small resource budget. The code interface allows for easy extensibility and addition of new features and models.\\n\\n## 1 Introduction\\n\\nConverting PDF documents back into a machine-processable format has been a major challenge for decades due to their huge variability in formats, weak standardization and printing-optimized characteristic, which discards most structural features and metadata. With the advent of LLMs and popular application patterns such as retrieval-augmented generation (RAG), leveraging the rich content embedded in PDFs has become ever more relevant. In the past decade, several powerful document understanding solutions have emerged on the market, most of which are commercial software, cloud offerings [3] and most recently, multi-modal vision-language models. As of today, only a handful of open-source tools cover PDF conversion, leaving a significant feature and quality gap to proprietary solutions.\\n\\nWith Docling , we open-source a very capable and efficient document conversion tool which builds on the powerful, specialized AI models and datasets for layout analysis and table structure recognition we developed and presented in the recent past [12, 13, 9]. Docling is designed as a simple, self-contained python library with permissive license, running entirely locally on commodity hardware. Its code architecture allows for easy extensibility and addition of new features and models.\\n\\nHere is what Docling delivers today:\\n\\n- Converts PDF documents to JSON or Markdown format, stable and lightning fast\\n- Understands detailed page layout, reading order, locates figures and recovers table structures\\n- Extracts metadata from the document, such as title, authors, references and language\\n- Optionally applies OCR, e.g. for scanned PDFs\\n- Can be configured to be optimal for batch-mode (i.e high throughput, low time-to-solution) or interactive mode (compromise on efficiency, low time-to-solution)\\n- Can leverage different accelerators (GPU, MPS, etc).\\n\\n## 2 Getting Started\\n\\nTo use Docling, you can simply install the docling package from PyPI. Documentation and examples are available in our GitHub repository at github.com/DS4SD/docling. All required model assets 1 are downloaded to a local huggingface datasets cache on first use, unless you choose to pre-install the model assets in advance.\\n\\nDocling provides an easy code interface to convert PDF documents from file system, URLs or binary streams, and retrieve the output in either JSON or Markdown format. For convenience, separate methods are offered to convert single documents or batches of documents. A basic usage example is illustrated below. Further examples are available in the Doclign code repository.\\n\\nfrom docling.document\\\\_converter import DocumentConverter\\n\\n```\\nsource = \\\"https://arxiv.org/pdf/2206.01062\\\" # PDF path or URL converter = DocumentConverter() result = converter.convert_single(source) print(result.render_as_markdown()) # output: \\\"## DocLayNet: A Large Human -Annotated Dataset for Document -Layout Analysis [...]\\\"\\n```\\n\\nOption

-- Tool output: {"type":"text","text":"{\n  \"from_cache\": false,\n  \"document_key\": \"994956e127ac4fc83092023df93360da\"\n}","annotations":null,"meta":null}

🤖 Assistant>

Summary of “Docling Technical Report” (arXiv 2408.09869)

What is Docling?
Docling is an open‑source (MIT‑licensed) Python library for converting PDF documents into structured, machine‑readable formats (JSON or Markdown). It runs locally on commodity hardware and is designed to be fast, stable, and easy to extend.

Key Capabilities

                                                                                                    
  Feature                       Description                                                         
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  Layout analysis               Uses a custom object‑detector (RT‑DETR‑based) trained on the        
                                large, human‑annotated DocLayNet dataset to identify page elements  
                                (paragraphs, headings, lists, figures, tables, etc.).               
  Table structure recognition   Employs TableFormer, a vision‑transformer model that recovers       
                                logical row/column structures, spanning cells, headers, and         
                                hierarchies.                                                        
  OCR (optional)                Integrates EasyOCR for scanned PDFs (high‑resolution 216 dpi        
                                input).                                                             
  Metadata extraction           Detects title, authors, references, language, and other             
                                bibliographic information.                                          
  Configurable pipelines        Users can enable/disable OCR, table recognition, set thread         
                                budgets, and choose GPU/MPS acceleration.                           
  Extensibility                 The model pipeline is modular; developers can subclass the base     
                                pipeline to add new models (e.g., figure classifier, equation       
                                recognizer).                                                        
  Batch & interactive modes     Optimized for high‑throughput batch processing or low‑latency       
                                interactive use.                                                    
                                                                                                    

Processing Pipeline

 1 PDF backend – extracts text tokens with coordinates and renders page images. Two backends are off
 2 AI model stage – runs layout analysis and table‑structure models on each page image.             
 3 Post‑processing – merges predictions, fixes reading order, adds language detection, and assembles
 4 Serialization – outputs JSON or Markdown.                                                        

Performance (reference hardware)

                                                                                                    
                                             Time‑to‑solution       Throughput                      
  System                Threads   Backend    (225 pages)            (pages/s)             Peak RAM  
 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 
  Apple M3 Max (16      4         native     177 s                  1.27                  6.2 GB    
  cores)                                                                                            
  Apple M3 Max (16      16        native     167 s                  1.34                  6.2 GB    
  cores)                                                                                            
  Apple M3 Max (16      4         pypdfium   103 s                  2.18                  2.6 GB    
  cores)                                                                                            
  Apple M3 Max (16      16        pypdfium   92 s                   2.45 

### Agentic RAG

For large document, or in general for a corpus of documents, we can leverage the power of the Docling document conversion with Agentic RAG.
During the ingestion, thanks to Docling, we will ingest the textual document components as well as the more complex structures like tables and figures.
While the Agentic RAG enhance the usual RAG flows thanks to 1) having the model rephrasing the retrieval query and 2) allowing the reasoning loop to decide if the retrieved context are good and enough or a second or third retrieval iteration is needed.

In [8]:
# Create a vectordb index in Llama Stack
vdb_id_resp = lls_client.vector_dbs.register(
    vector_db_id=(vdb_name := f"testdb_{uuid.uuid4()}"),
    embedding_model="sentence-transformers/all-MiniLM-L6-v2",
    embedding_dimension=384,
    provider_id="milvus",
)
vdb_id = vdb_id_resp.identifier

# Instruct the agent to ingest the data in the index and use it for querying.
await run_agent(
    queries=[
        f"Ingest the document https://arxiv.org/pdf/2206.01062 into the vectordb {vdb_id} and answer the following questions: 1) how many pages were manually annotated?",
    ],
    extra_tools=[
        FileSearchTool(vector_store_ids=[vdb_id])
    ],
)


INFO:mcp.client.streamable_http:Received session ID: 67912c5851164b08ba8851209b50dc49
INFO:mcp.client.streamable_http:Negotiated protocol version: 2025-06-18


Created session_id=6926a2df-c96f-4deb-96ea-1e82fe75a2c2 for Agent(Document Agent)


👤 User> You are an assistant that uses external tools.  
Follow these rules strictly:

1. At each reasoning step, you may call **at most one tool**.  
2. If you need to use multiple tools, do it sequentially:  
   - Call exactly one tool.  
   - Wait for its result.  
   - Then think again and, if needed, call another tool.  
3. Never call two or more tools in the same response.  
4. If no tool is needed, just provide your reasoning or final answer.  
5. Treat a missing tool result as blocking — do not continue reasoning until you receive the output.

If you ever attempt to call more than one tool at once, your answer will be rejected.  
Always restrict yourself to **zero or one tool call per response**.

When the tools return a base64 image, simply create some markdown wrapper which allows to display it and take special care in not hallucinating any byte.

Agent updated: Document Agent

🤖 Assistant>

...

-- Tool output: {"type":"text","text":"{\n  \"document_key\": \"868f49ae1f0e66e82238a8aea43fd30b\",\n  \"markdown\": \"## DocLayNet: A Large Human-Annotated Dataset for Document-Layout Analysis\\n\\nBirgit Pfitzmann IBM Research Rueschlikon, Switzerland bpf@zurich.ibm.com\\n\\nChristoph Auer IBM Research Rueschlikon, Switzerland cau@zurich.ibm.com\\n\\nAhmed S. Nassar IBM Research\\n\\nRueschlikon, Switzerland ahn@zurich.ibm.com\\n\\nMichele Dolfi IBM Research Rueschlikon, Switzerland dol@zurich.ibm.com\\n\\nPeter Staar IBM Research Rueschlikon, Switzerland taa@zurich.ibm.com\\n\\nFigure 1: Four examples of complex page layouts across different document categories\\n\\n<!-- image -->\\n\\n## KEYWORDS\\n\\nPDF document conversion, layout segmentation, object-detection, data set, Machine Learning\\n\\n## ACMReference Format:\\n\\nBirgit Pfitzmann, Christoph Auer, Michele Dolfi, Ahmed S. Nassar, and Peter Staar. 2022. DocLayNet: A Large Human-Annotated Dataset for DocumentLayout Analysis. In Proceedings of the 28th ACM SIGKDD Conference on Knowledge Discovery and Data Mining (KDD '22), August 14-18, 2022, Washington, DC, USA. ACM, New York, NY, USA, 9 pages. https://doi.org/10.1145/ 3534678.3539043\\n\\n## ABSTRACT\\n\\nAccurate document layout analysis is a key requirement for highquality PDF document conversion. With the recent availability of public, large ground-truth datasets such as PubLayNet and DocBank, deep-learning models have proven to be very effective at layout detection and segmentation. While these datasets are of adequate size to train such models, they severely lack in layout variability since they are sourced from scientific article repositories such as PubMed and arXiv only. Consequently, the accuracy of the layout segmentation drops significantly when these models are applied on more challenging and diverse layouts. In this paper, we present DocLayNet , a new, publicly available, document-layout annotation dataset in COCO format. It contains 80863 manually annotated pages from diverse data sources to represent a wide variability in layouts. For each PDF page, the layout annotations provide labelled bounding-boxes with a choice of 11 distinct classes. DocLayNet also provides a subset of double- and triple-annotated pages to determine the inter-annotator agreement. In multiple experiments, we provide baseline accuracy scores (in mAP) for a set of popular object detection models. We also demonstrate that these models fall approximately 10% behind the inter-annotator agreement. Furthermore, we provide evidence that DocLayNet is of sufficient size. Lastly, we compare models trained on PubLayNet, DocBank and DocLayNet, showing that layout predictions of the DocLayNettrained models are more robust and thus the preferred choice for general-purpose document-layout analysis.\\n\\n## CCS CONCEPTS\\n\\n· Informationsystems → Documentstructure ; · Appliedcomputing → Document analysis ; · Computing methodologies → Machine learning ; Computer vision ; Object detection ;\\n\\nPermission to make digital or hard copies of part or all of this work for personal or classroom use is granted without fee provided that copies are not made or distributed for profit or commercial advantage and that copies bear this notice and the full citation on the first page. Copyrights for third-party components of this work must be honored. For all other uses, contact the owner/author(s).\\n\\nKDD '22, August 14-18, 2022, Washington, DC, USA\\n\\n© 2022 Copyright held by the owner/author(s).\\n\\nACM ISBN 978-1-4503-9385-0/22/08.\\n\\nhttps://doi.org/10.1145/3534678.3539043\\n\\nKDD '22, August 14-18, 2022, Washington, DC, USA Birgit Pfitzmann, Christoph Auer, Michele Dolfi, Ahmed S. Nassar, and Peter Staar\\n\\n## 1 INTRODUCTION\\n\\nDespite the substantial improvements achieved with machine-learning (ML) approaches and deep neural networks in recent years, document conversion remains a challenging problem, as demonstrated by the numerous public c

-- Tool output: {"type":"text","text":"{\n  \"md_file\": \"/Users/dol/projects/tx25/techxchange2025-lab3640-docling/.venv/lib/python3.12/site-packages/_cache/868f49ae1f0e66e82238a8aea43fd30b.md\",\n  \"json_file\": \"/Users/dol/projects/tx25/techxchange2025-lab3640-docling/.venv/lib/python3.12/site-packages/_cache/868f49ae1f0e66e82238a8aea43fd30b.json\"\n}","annotations":null,"meta":null}

-- Tool output: {"type":"text","text":"{\n  \"from_cache\": true,\n  \"document_key\": \"868f49ae1f0e66e82238a8aea43fd30b\"\n}","annotations":null,"meta":null}

ERROR:openai.agents:Error streaming response: 500: Internal server error: An unexpected error occurred.


APIError: 500: Internal server error: An unexpected error occurred.

---

## 2. Create a document

Another interesting use-case of the Docling MCP tools is the possibility to have a model writing complex document directly in the DoclingDocument format which can then be exported to a variety of outputs (e.g. Markdown, HTML, Latex, etc).

In the following example we only provide the instructions on the topic and the key structure of the document and let the agent build it.

In [9]:
await run_agent(
    queries=[
        "Create a new Docling document with the title \"Open-Source Agentic AI\", a paragraph, and a list of the main applications.",
    ],
    extra_tools=[],
)

INFO:mcp.client.streamable_http:Received session ID: 884d1f7194454cde91b2ddd5d6616b6c
INFO:mcp.client.streamable_http:Negotiated protocol version: 2025-06-18


Created session_id=6e5d1dab-0c11-46ce-88ed-6f7bb9bfcecb for Agent(Document Agent)


👤 User> You are an assistant that uses external tools.  
Follow these rules strictly:

1. At each reasoning step, you may call **at most one tool**.  
2. If you need to use multiple tools, do it sequentially:  
   - Call exactly one tool.  
   - Wait for its result.  
   - Then think again and, if needed, call another tool.  
3. Never call two or more tools in the same response.  
4. If no tool is needed, just provide your reasoning or final answer.  
5. Treat a missing tool result as blocking — do not continue reasoning until you receive the output.

If you ever attempt to call more than one tool at once, your answer will be rejected.  
Always restrict yourself to **zero or one tool call per response**.

When the tools return a base64 image, simply create some markdown wrapper which allows to display it and take special care in not hallucinating any byte.

Agent updated: Document Agent

🤖 Assistant>

...

-- Tool output: {"type":"text","text":"{\n  \"document_key\": \"f1a9f12563db40c8906ae369d3636214\"\n}","annotations":null,"meta":null}

-- Tool was called: export_docling_document_to_markdown({
  "document_key": "f1a9f12563db40c8906ae369d3636214",
  "max_size": null
})

-- Tool output: {"type":"text","text":"{\n  \"document_key\": \"f1a9f12563db40c8906ae369d3636214\",\n  \"markdown\": \"# Open-Source Agentic AI\\n\\nOpen-Source Agentic AI refers to community-driven, self-governing artificial intelligence systems that can make decisions and act autonomously to achieve defined goals. These agents are built on open-source frameworks, enabling transparency, collaboration, and rapid innovation across diverse domains.\\n\\n- Autonomous research assistants\\n- Personal productivity bots\\n- Adaptive tutoring systems\\n- Automated code generation\\n- Decision support agents\"\n}","annotations":null,"meta":null}

🤖 Assistant>

Here is the Docling document you requested, rendered in markdown:

                                                                                                    
 # Open-Source Agentic AI                                                                           
                                                                                                    
 Open-Source Agentic AI refers to community-driven, self-governing artificial intelligence systems  
                                                                                                    
 - Autonomous research assistants                                                                   
 - Personal productivity bots                                                                       
 - Adaptive tutoring systems                                                                        
 - Automated code generation                                                                        
 - Decision support agents                                                                          
                                                                                                    


---

## Next Steps: Where to Go from Here

### Immediate actions

1. **Experiment with your documents**
   - Try documents with complex layouts
   - Test with technical diagrams and charts
   - Process multi-page reports with mixed content

2. **Connect more agents**
   - Try connecting more tools
   - Search the documents to ingest via metadata
   - Search the web for relevant documents
   - Extract information from the documents

3. **More ways to interact with tools**
   - Use the Llama Stack playground UI for chatting with the agents
   - Use other frameworks and ecosystems like Claude Desktop, BeeAI, etc

---

## Resources for Continued Learning

### Official Documentation
- **[Docling Documentation](https://github.com/docling-project/docling)**: Latest features and updates

### Community Resources
- Join the Docling community on GitHub
- Share your implementations
- Contribute improvements back to the project

### Related Topics to Explore
- Document Layout Analysis
- Multimodal Embeddings
- Visual Question Answering
- Explainable AI Systems

---